In [4]:
# Construct pandas dataframe
import numpy as np
import pandas as pd

df = pd.read_pickle("data.pkl")

In [7]:
df[0:10]

,subject,exp,phasename,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,values
0,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[134.35545018549945, 134.36062545137696, 134.3..."
1,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[130.2177785784425, 130.1799779128927, 130.066..."
2,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[132.48693891436434, 132.47957291468927, 132.4..."
3,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[139.03715025155026, 139.05524237006566, 139.1..."
4,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[130.9270407865815, 130.91305929142686, 130.87..."
5,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[112.6256623125161, 112.64933828213299, 112.72..."
6,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[123.21815824513973, 123.244447415985, 123.323..."
7,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[139.2634701481919, 139.26678851999134, 139.27..."
8,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[145.61829050782887, 145.62492057966884, 145.6..."
9,33,EC,3,1,2,3166,30,HECK,I,20170301,1,1,"[124.04958865511853, 124.04394652482138, 124.0..."


In [10]:
len(df.values)

44012

In [23]:
df.loc['values']

subject                                                       33
exp                                                           EC
phasename                                                      3
num_rep                                                        1
num_trial                                                      2
num_formed                                                  3166
num_uttered                                                   30
prompt                                                      HECK
in_or_out                                                      I
date_run                                                20170301
time_run                                                       1
block                                                          1
values         [130.2177785784425, 130.1799779128927, 130.066...
Name: 1, dtype: object